In [39]:

import boto3
import pandas as pd
from pandas import json_normalize
import ast
import json
from decimal import Decimal
from datetime import date
from datetime import timedelta
import time
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import gc
from keras.callbacks import Callback
import pickle

In [36]:
pd.set_option('display.max_columns', None)

In [ ]:


# Configure your AWS credentials and region
aws_access_key_id = super_secrets.Access_key_ID
aws_secret_access_key = super_secrets.Secret_access_key
aws_region = super_secrets.aws_region

table_name='nhl_events'

# Initialize the DynamoDB client
dynamodb = boto3.resource('dynamodb',
                          aws_access_key_id=aws_access_key_id,
                          aws_secret_access_key=aws_secret_access_key,
                          region_name=aws_region)

# Get the table object
table = dynamodb.Table(table_name)




In [ ]:
def scan_table(table):
    items = []
    response = None
    while response is None or 'LastEvaluatedKey' in response:
        if response is not None and 'LastEvaluatedKey' in response:
            response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
        else:
            response = table.scan()
        items.extend(response['Items'])
    return items

In [ ]:
def safe_eval(s):
    try:
        return eval(s)
    except Exception as e:
        print(f"Error evaluating string: {s}. Error: {e}")
        return None

In [ ]:
def convert_decimal_to_float(x):
    if isinstance(x, Decimal):
        return float(x)
    return x

In [ ]:
items = scan_table(table)
df = pd.DataFrame(items)
df.to_csv('out.csv', index=False)
#df = pd.read_csv('out.csv')

In [ ]:
df['result'] = df['result'].apply(safe_eval)
df['about'] = df['about'].apply(safe_eval)
df['team'] = df['team'].apply(safe_eval)
df['coordinates'] = df['coordinates'].apply(safe_eval)

In [ ]:
result = df['result'].apply(pd.Series)
about = df['about'].apply(pd.Series)
team = df['team'].apply(pd.Series)
coordinates = df['coordinates'].apply(pd.Series)

In [ ]:
df = df.join(result)
df = df.join(about)
df = df.join(team)
df = df.join(coordinates)

In [ ]:
df = df.drop(columns=['result','about','team','coordinates','players','link','eventCode','event','gameWinningGoal','eventId','periodType','ordinalNum','name'])


In [ ]:
goals = df['goals'].apply(pd.Series)
goals.rename(columns={'away':'goals_away','home':'goals_home'},inplace=True)
df = df.join(goals)
df = df.drop(columns=['goals'])

In [ ]:
df=df.fillna(0)

In [ ]:
df.to_csv('cleaned.csv', index=False,)

In [ ]:
def time_string_to_minutes(time_string):
    minutes, seconds = map(int, time_string.split(':'))
    return minutes * 60 + seconds

def datetime_to_minutes(datetime_obj):
    unix_timestamp = datetime_obj.timestamp()
    return int(unix_timestamp // 60)

In [ ]:
df['time_left_in_period'] = df['periodTimeRemaining'].apply(time_string_to_minutes)
df['dateTime'] = pd.to_datetime(df['dateTime'])
df['epoch_time'] = df['dateTime'].apply(lambda dt: int(dt.timestamp()))

In [ ]:
df = df.drop(columns=['periodTime','periodTimeRemaining'])
df['Home_Shots']=0
df['Away_Shots']=0
df['Home_Faceoffs']=0
df['Away_Faceoffs']=0
df['Home_blocked_shots']=0
df['Away_blocked_shots']=0
df['Home_hits']=0
df['Away_hits']=0
df['Home_takeaway']=0
df['Away_takeaway']=0
df['Home_giveaway']=0
df['Away_giveaway']=0
df['Home_penalty_minutes']=0
df['Away_penalty_minutes']=0

In [ ]:
df = df.applymap(convert_decimal_to_float)

In [ ]:
games_id=df["Game_ID"].unique()

In [ ]:
games_id=df["eventTypeId"].unique()
games_id

In [ ]:
column_names = df.columns.tolist()
new_df=pd.DataFrame(columns=column_names)



counter4=0
for i in games_id:
    start_time = time.time()
    games_df=df.loc[df['Game_ID'] == i].copy()
    games_df.sort_values(by=['eventIdx'], inplace=True)
    games_df.reset_index(drop=True, inplace=True)
    counter4+=1
    
    
    
    for index, row in games_df.iterrows():
        
        if counter4 >= len(games_id):
            break
        else:
            
            if row['eventIdx'] == 0:
                pass
            else:
                event_id=row['eventIdx']
                previous = games_df.loc[index-1]
                Homeshot=previous['Home_Shots']
                Awayshot=previous['Away_Shots']
                Home_faceoff=previous['Home_Faceoffs']
                Away_faceoff=previous['Away_Faceoffs']
                Home_Blocked=previous['Home_blocked_shots']
                Away_Blocked=previous['Away_blocked_shots']
                Home_Hits=previous['Home_hits']
                Away_Hits=previous['Away_hits']
                Home_tw=previous['Home_takeaway']
                Away_tw=previous['Away_takeaway']
                Home_gw=previous['Home_giveaway']
                Away_gw=previous['Away_giveaway']
                Home_pim=previous['Home_penalty_minutes']
                Away_pim=previous['Away_penalty_minutes']
                if (row['eventTypeId']) == 'SHOT' or (row['eventTypeId']) == 'MISSED_SHOT':
                    if row['id'] == row['home']:
                        Homeshot+=1
                    else:
                        Awayshot+=1

                elif (row['eventTypeId']) == 'FACEOFF':
                    if row['id'] == row['home']:
                        Home_faceoff+=1
                    else:
                        Away_faceoff+=1 

                elif (row['eventTypeId']) == 'BLOCKED_SHOT':
                    if row['id'] == row['home']:
                        Home_Blocked+=1
                    else:
                        Away_Blocked+=1  

                elif (row['eventTypeId']) == 'TAKEAWAY':
                    if row['id'] == row['home']:
                        Home_tw+=1
                    else:
                        Away_tw+=1  

                elif (row['eventTypeId']) == 'HIT':
                    if row['id'] == row['home']:
                        Home_Hits+=1
                    else:
                        Away_Hits+=1  

                elif (row['eventTypeId']) == 'GIVEAWAY':
                    if row['id'] == row['home']:
                        Home_gw+=1
                    else:
                        Away_gw+=1

                elif (row['eventTypeId']) == 'PENALTY':
                    if row['id'] == row['home']:
                        Home_pim+=row['penaltyMinutes']
                    else:
                        Away_pim+=row['penaltyMinutes']

                else:
                    pass
                games_df.at[index,'Home_Shots']=Homeshot
                games_df.at[index,'Away_Shots']=Awayshot
                games_df.at[index,'Home_Faceoffs']=Home_faceoff
                games_df.at[index,'Away_Faceoffs']=Away_faceoff
                games_df.at[index,'Home_blocked_shots']=Home_Blocked
                games_df.at[index,'Away_blocked_shots']=Away_Blocked
                games_df.at[index,'Home_hits']=Home_Hits
                games_df.at[index,'Away_hits']=Away_Hits
                games_df.at[index,'Home_takeaway']=Home_tw
                games_df.at[index,'Away_takeaway']=Away_tw
                games_df.at[index,'Home_giveaway']=Home_gw
                games_df.at[index,'Away_giveaway']=Away_gw
                games_df.at[index,'Home_penalty_minutes']=Home_pim
                games_df.at[index,'Away_penalty_minutes']=Away_pim
    new_df=pd.concat([new_df, games_df])
        
    #new_df=pd.concat([new_df, games_df])
    end_time = time.time()
    cycle_time = end_time - start_time
    print(f"Cycle {counter4} took {cycle_time:.4f} seconds")
 

In [15]:
#new_df.to_csv('cleaned_added_columns.csv', index=False)
new_df = pd.read_csv('cleaned_added_columns.csv')

In [16]:
new_df.sort_values(by=['Game_ID'], inplace=True)
new_df.head()

,season,Home_Win_Bool,UID,game_type,away,Game_ID,home,secondaryType,description,eventTypeId,penaltyMinutes,penaltySeverity,strength,emptyNet,eventIdx,dateTime,period,id,triCode,x,y,goals_away,goals_home,time_left_in_period,epoch_time,Home_Shots,Away_Shots,Home_Faceoffs,Away_Faceoffs,Home_blocked_shots,Away_blocked_shots,Home_hits,Away_hits,Home_takeaway,Away_takeaway,Home_giveaway,Away_giveaway,Home_penalty_minutes,Away_penalty_minutes
224640,20172018,1,2017020223_239,R,15,2017020223,7,0,Dmitry Orlov hit Justin Falk,HIT,0.0,0,0,0,239.0,2017-11-08 02:16:05+00:00,3.0,15.0,WSH,97.0,-38.0,1.0,2.0,577,1510107365,37,23,24,25,7,8,15,18,4,5,7,7,7,13
224689,20172018,1,2017020223_288,R,15,2017020223,7,0,Game End,GAME_END,0.0,0,0,0,288.0,2017-11-08 02:35:22+00:00,3.0,0.0,0,0.0,0.0,1.0,3.0,0,1510108522,39,30,31,29,12,8,19,22,6,5,7,7,7,13
224591,20172018,1,2017020223_190,R,15,2017020223,7,0,Goalie Stopped,STOP,0.0,0,0,0,190.0,2017-11-08 01:39:08+00:00,2.0,0.0,0,0.0,0.0,1.0,1.0,27,1510105148,29,17,19,18,7,6,14,14,3,4,7,7,7,9
224590,20172018,1,2017020223_189,R,15,2017020223,7,Wrist Shot,John Carlson Wrist Shot saved by Robin Lehner,SHOT,0.0,0,0,0,189.0,2017-11-08 01:39:06+00:00,2.0,15.0,WSH,43.0,-35.0,1.0,1.0,29,1510105146,29,17,19,18,7,6,14,14,3,4,7,7,7,9
224589,20172018,1,2017020223_188,R,15,2017020223,7,0,Ryan O'Reilly faceoff won against Alex Chiasson,FACEOFF,0.0,0,0,0,188.0,2017-11-08 01:38:44+00:00,2.0,7.0,BUF,69.0,-22.0,1.0,1.0,35,1510105124,29,16,19,18,7,6,14,14,3,4,7,7,7,9


In [17]:
df_model=new_df[['Home_Win_Bool','season','game_type','away','home',
                 'penaltyMinutes','penaltySeverity','eventIdx','period',
                 'x','y','goals_away','goals_home',
                 'time_left_in_period','epoch_time','Home_Shots','Away_Shots','Home_Faceoffs',
                'Away_Faceoffs','Home_blocked_shots','Away_blocked_shots',
                'Home_hits','Away_hits','Home_takeaway','Away_takeaway','Home_giveaway','Away_giveaway',
                'Home_penalty_minutes','Away_penalty_minutes']]

In [53]:
d_model_encoded=pd.get_dummies(df_model,columns=[
                                                 'penaltySeverity','home','away','game_type'], dtype=float)

In [19]:
d_model_encoded.to_csv('d_model_encoded5.csv', index=False)

In [54]:
empty_df = pd.DataFrame(columns=d_model_encoded.columns)
empty_df.to_csv('empty_df.csv', index=False)

In [55]:
d_model_encoded.head()

,Home_Win_Bool,season,penaltyMinutes,eventIdx,period,x,y,goals_away,goals_home,time_left_in_period,epoch_time,Home_Shots,Away_Shots,Home_Faceoffs,Away_Faceoffs,Home_blocked_shots,Away_blocked_shots,Home_hits,Away_hits,Home_takeaway,Away_takeaway,Home_giveaway,Away_giveaway,Home_penalty_minutes,Away_penalty_minutes,penaltySeverity_0,penaltySeverity_Bench Minor,penaltySeverity_Game Misconduct,penaltySeverity_Major,penaltySeverity_Match,penaltySeverity_Minor,penaltySeverity_Misconduct,penaltySeverity_Penalty Shot,home_1,home_2,home_3,home_4,home_5,home_6,home_7,home_8,home_9,home_10,home_12,home_13,home_14,home_15,home_16,home_17,home_18,home_19,home_20,home_21,home_22,home_23,home_24,home_25,home_26,home_28,home_29,home_30,home_52,home_53,home_54,home_55,home_7461,away_1,away_2,away_3,away_4,away_5,away_6,away_7,away_8,away_9,away_10,away_12,away_13,away_14,away_15,away_16,away_17,away_18,away_19,away_20,away_21,away_22,away_23,away_24,away_25,away_26,away_28,away_29,away_30,away_52,away_53,away_54,away_55,away_7460,game_type_P,game_type_R,game_type_WA
224640,1,20172018,0.0,239.0,3.0,97.0,-38.0,1.0,2.0,577,1510107365,37,23,24,25,7,8,15,18,4,5,7,7,7,13,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
224689,1,20172018,0.0,288.0,3.0,0.0,0.0,1.0,3.0,0,1510108522,39,30,31,29,12,8,19,22,6,5,7,7,7,13,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
224591,1,20172018,0.0,190.0,2.0,0.0,0.0,1.0,1.0,27,1510105148,29,17,19,18,7,6,14,14,3,4,7,7,7,9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
224590,1,20172018,0.0,189.0,2.0,43.0,-35.0,1.0,1.0,29,1510105146,29,17,19,18,7,6,14,14,3,4,7,7,7,9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
224589,1,20172018,0.0,188.0,2.0,69.0,-22.0,1.0,1.0,35,1510105124,29,16,19,18,7,6,14,14,3,4,7,7,7,9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [56]:
d_model_encoded.tail()

,Home_Win_Bool,season,penaltyMinutes,eventIdx,period,x,y,goals_away,goals_home,time_left_in_period,epoch_time,Home_Shots,Away_Shots,Home_Faceoffs,Away_Faceoffs,Home_blocked_shots,Away_blocked_shots,Home_hits,Away_hits,Home_takeaway,Away_takeaway,Home_giveaway,Away_giveaway,Home_penalty_minutes,Away_penalty_minutes,penaltySeverity_0,penaltySeverity_Bench Minor,penaltySeverity_Game Misconduct,penaltySeverity_Major,penaltySeverity_Match,penaltySeverity_Minor,penaltySeverity_Misconduct,penaltySeverity_Penalty Shot,home_1,home_2,home_3,home_4,home_5,home_6,home_7,home_8,home_9,home_10,home_12,home_13,home_14,home_15,home_16,home_17,home_18,home_19,home_20,home_21,home_22,home_23,home_24,home_25,home_26,home_28,home_29,home_30,home_52,home_53,home_54,home_55,home_7461,away_1,away_2,away_3,away_4,away_5,away_6,away_7,away_8,away_9,away_10,away_12,away_13,away_14,away_15,away_16,away_17,away_18,away_19,away_20,away_21,away_22,away_23,away_24,away_25,away_26,away_28,away_29,away_30,away_52,away_53,away_54,away_55,away_7460,game_type_P,game_type_R,game_type_WA
1044731,0,20222023,0.0,279.0,3.0,92.0,29.0,4.0,3.0,601,1683086418,33,41,25,22,16,11,12,32,3,10,12,6,6,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1044732,0,20222023,0.0,280.0,3.0,63.0,25.0,4.0,3.0,601,1683086418,33,41,25,22,17,11,12,32,3,10,12,6,6,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1044733,0,20222023,0.0,281.0,3.0,58.0,41.0,4.0,3.0,585,1683086434,33,41,25,22,17,11,12,32,3,10,13,6,6,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1044723,0,20222023,0.0,271.0,3.0,74.0,0.0,4.0,2.0,702,1683086113,32,38,25,21,15,11,12,32,3,10,11,6,6,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2386586,0,20222023,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1200,1683161246,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [57]:
print(d_model_encoded.shape)

(2386587, 102)


In [58]:
print(d_model_encoded.dtypes)

Home_Win_Bool       int64
season              int64
penaltyMinutes    float64
eventIdx          float64
period            float64
                   ...   
away_55           float64
away_7460         float64
game_type_P       float64
game_type_R       float64
game_type_WA      float64
Length: 102, dtype: object


In [59]:
print(d_model_encoded.describe())

       Home_Win_Bool        season  penaltyMinutes      eventIdx   
count   2.386587e+06  2.386587e+06    2.386587e+06  2.386587e+06  \
mean    5.365101e-01  2.019753e+07    5.420628e-02  1.622692e+02   
std     4.986653e-01  1.740819e+04    4.105527e-01  9.646647e+01   
min     0.000000e+00  2.017202e+07    0.000000e+00  0.000000e+00   
25%     0.000000e+00  2.018202e+07    0.000000e+00  8.000000e+01   
50%     1.000000e+00  2.019202e+07    0.000000e+00  1.600000e+02   
75%     1.000000e+00  2.021202e+07    0.000000e+00  2.410000e+02   
max     1.000000e+00  2.022202e+07    1.000000e+01  7.830000e+02   

             period             x             y    goals_away    goals_home   
count  2.386587e+06  2.386587e+06  2.386587e+06  2.386587e+06  2.386587e+06  \
mean   2.019204e+00 -1.002532e-01 -4.257042e-02  1.332097e+00  1.453051e+00   
std    8.612989e-01  5.802160e+01  2.051579e+01  1.377671e+00  1.476330e+00   
min    1.000000e+00 -1.000000e+02 -4.200000e+01  0.000000e+00  0.000000

In [60]:
print(d_model_encoded.isnull().sum())

Home_Win_Bool     0
season            0
penaltyMinutes    0
eventIdx          0
period            0
                 ..
away_55           0
away_7460         0
game_type_P       0
game_type_R       0
game_type_WA      0
Length: 102, dtype: int64


In [27]:
#del [[df,new_df,df_model]]
#gc.collect()

NameError: name 'df' is not defined

In [37]:
d_model_encoded = pd.read_csv('d_model_encoded3.csv')

In [61]:
d_model_encoded['Home_Win_Bool']=d_model_encoded['Home_Win_Bool'].astype(int)
d_model_encoded['season']=d_model_encoded['season'].astype(int)

In [62]:
d_model_encoded = d_model_encoded.sort_index(axis=1)

train=d_model_encoded.head(int(len(d_model_encoded.index)*.80))
best=d_model_encoded.tail(int(len(d_model_encoded.index)*.05))
validation=d_model_encoded.head(int(len(best.index)*.50))
test=d_model_encoded.tail(int(len(best.index)*.50))

train=train.sample(frac=1)

In [63]:
train_x=train.drop('Home_Win_Bool',axis=1)
train_y=train['Home_Win_Bool']
test_x=test.drop('Home_Win_Bool',axis=1)
test_y=test['Home_Win_Bool']
val_x=validation.drop('Home_Win_Bool',axis=1)
val_y=validation['Home_Win_Bool']

In [64]:
#train.to_csv('train_x.csv', index=False)

train_columns = pd.DataFrame(columns=train_x.columns)
train_columns.to_csv('train_df.csv', index=False)

In [65]:
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)
val_x = scaler.transform(val_x)

with open("scalar_model.pkl", "wb") as file:
    pickle.dump(scaler, file)

In [ ]:
model = Sequential([
    Dense(train_x.shape[1], activation='relu', input_shape=(train_x.shape[1],)),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:

logs = Callback()
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=20, batch_size=2048,callbacks=[logs])
test_loss, test_accuracy = model.evaluate(test_x, test_y)
print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_accuracy:.4f}")

In [ ]:
eval=model.evaluate(
    x=test_x,
    y=test_y,
    batch_size=None,
    verbose='auto',
    sample_weight=None,
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
    return_dict=True,
   
)

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
y_pred_probs = model.predict(test_x)
y_pred = (y_pred_probs > 0.5).astype(int)

In [ ]:
for i in y_pred:
    print(i[0])

In [ ]:
y_pred_probs
p=[]
for i in y_pred_probs:
    print(np.around(i,decimals=4))
    p.append(i)
    

In [ ]:
p

In [ ]:
#logistic regression

In [66]:




# Create and train the logistic regression model
logreg = LogisticRegression()
logreg.fit(train_x, train_y)

# Make predictions using the testing set
y_pred = logreg.predict(test_x)

# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(test_y, y_pred))
print("\nClassification Report:")
print(classification_report(test_y, y_pred))
print("Accuracy Score:")
print(accuracy_score(test_y, y_pred))

Confusion Matrix:
[[20863  9744]
 [ 4893 24164]]

Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.68      0.74     30607
           1       0.71      0.83      0.77     29057

    accuracy                           0.75     59664
   macro avg       0.76      0.76      0.75     59664
weighted avg       0.76      0.75      0.75     59664

Accuracy Score:
0.7546761866452132


In [67]:
with open('logreg.pkl','wb') as file:
    pickle.dump(logreg, file)

In [ ]:
# Get an item by primary key
def get_item(primary_key_value):
    response = table.get_item(Key={'your_primary_key_name': primary_key_value})
    return response.get('Item', None)

# Update an item
def update_item(primary_key_value, update_expression, expression_attribute_values):
    response = table.update_item(
        Key={'your_primary_key_name': primary_key_value},
        UpdateExpression=update_expression,
        ExpressionAttributeValues=expression_attribute_values,
        ReturnValues="UPDATED_NEW"
    )
    return response

# Delete an item by primary key
def delete_item(primary_key_value):
    response = table.delete_item(Key={'your_primary_key_name': primary_key_value})
    return response

# Example usage
item = {
    'your_primary_key_name': 'example_id',
    'attribute1': 'value1',
    'attribute2': 'value2'
}

# Insert the item
print("Inserting item:")
insert_response = insert_item(item)
print(insert_response)

# Get the item
print("Getting item:")
get_response = get_item('example_id')
print(get_response)

# Update the item
print("Updating item:")
update_response = update_item(
    'example_id',
    'SET attribute1 = :value',
    {':value': 'updated_value1'}
)
print(update_response)

# Delete the item
print("Deleting item:")
delete_response = delete_item('example_id')
print(delete_response)
